In [4]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel

# Load CSV as pandas DataFrame
df = pd.read_csv("data.csv")

# Convert pandas dataframe to HuggingFace dataset
dataset = Dataset.from_pandas(df)

# 1. Define the path to your local model directory
model_path = "/content/drive/MyDrive/Colab Notebooks/Actionable-Fine-Tune/mobilebert-finetuned-actionable"

# 2. Load tokenizer from the local directory
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Tokenize function
def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# Tokenize dataset
dataset = dataset.map(tokenize_fn, batched=True)


Map:   0%|          | 0/1301 [00:00<?, ? examples/s]

In [5]:
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",   # Evaluate at end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
)


In [8]:
from transformers import Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)


In [9]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pranjalpravesh121 (pranjalpravesh121-iit-delhi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.565500,0.158318,0.938697,0.931034,0.907563,0.955752
2,0.299000,0.109781,0.954023,0.945455,0.971963,0.920354
3,0.162500,0.106568,0.957854,0.949772,0.981132,0.920354


TrainOutput(global_step=195, training_loss=0.28436225255330405, metrics={'train_runtime': 101.9895, 'train_samples_per_second': 30.591, 'train_steps_per_second': 1.912, 'total_flos': 48912646348800.0, 'train_loss': 0.28436225255330405, 'epoch': 3.0})

In [10]:
trainer.evaluate()


{'eval_loss': 0.10656821727752686,
 'eval_accuracy': 0.9578544061302682,
 'eval_f1': 0.9497716894977168,
 'eval_precision': 0.9811320754716981,
 'eval_recall': 0.9203539823008849,
 'eval_runtime': 1.3875,
 'eval_samples_per_second': 188.104,
 'eval_steps_per_second': 12.252,
 'epoch': 3.0}

In [12]:
import shutil
from google.colab import files

# Define model directory
save_directory = "mobilebert-finetuned-actionable-v2"

# Save model and tokenizer
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

# Zip the directory
shutil.make_archive(save_directory, 'zip', save_directory)

# Download the zip file
files.download(f"{save_directory}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Test

In [15]:
import torch
from transformers import MobileBertForSequenceClassification, AutoTokenizer
import time

# Load model and tokenizer
model_path = "mobilebert-finetuned-actionable-v2"
model = MobileBertForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()

# Sample 25 sentences (this is actual data which were non actionable)
sentences = [
  "Control, control, control.",
  "And what is apparent towards this?",
  "You",
  "I’ll see you at the development tomorrow at one o’clock.",
  "I don’t know what my...",
  "Hmm",
  "Can you some more step forward?",
  "You responding with I wanted to ask about this sector of it on the card.",
  "What?",
  "Hello, hello, hello, hello.",
  "You know",
  "What is it?",
  "Okay, I’ll accept if this is funny.",
  "I’ll see you in the next video. Bye.",
  "Hello?",
  "I should do the meeting.",
  "Again if I say anything random it is now it is going to capture my audio but and also it is going to transcribe it but it the actionable gate is going to filter all of the unnecessary part out and only the necessary part will be translated and",
  "Will be processed by LLM.",
  "Repository links direct access to your fine-tuned model on having this",
  "Links for access to.",
  "Bye, Joe.",
  "So this is currently",
  "Wait.",
  "The meaning of",
  "Yes.",
  "Multiplayer. Hi, for God’s sake.",
  "I apologize.",
  "Have a good flight."
]

# Tokenize all at once (batch)
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

# Measure inference time
start_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

end_time = time.time()
elapsed_time = end_time - start_time

# Display results
for sentence, pred in zip(sentences, predictions):
    print(f"Text: {sentence}\nPredicted Label: {pred.item()}\n")

print(f"\nTotal inference time for 25 sentences: {elapsed_time:.4f} seconds")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text: Control, control, control.
Predicted Label: 0

Text: And what is apparent towards this?
Predicted Label: 0

Text: You
Predicted Label: 0

Text: I’ll see you at the development tomorrow at one o’clock.
Predicted Label: 0

Text: I don’t know what my...
Predicted Label: 0

Text: Hmm
Predicted Label: 0

Text: Can you some more step forward?
Predicted Label: 0

Text: You responding with I wanted to ask about this sector of it on the card.
Predicted Label: 0

Text: What?
Predicted Label: 0

Text: Hello, hello, hello, hello.
Predicted Label: 0

Text: You know
Predicted Label: 0

Text: What is it?
Predicted Label: 0

Text: Okay, I’ll accept if this is funny.
Predicted Label: 0

Text: I’ll see you in the next video. Bye.
Predicted Label: 0

Text: Hello?
Predicted Label: 0

Text: I should do the meeting.
Predicted Label: 0

Text: Again if I say anything random it is now it is going to capture my audio but and also it is going to transcribe it but it the actionable gate is going to filter a